In [1]:
import sys
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import json
import cohere
import gradio as gr
from keybert import KeyBERT
from transformers import AutoModel, AutoTokenizer
from langchain.schema.document import Document
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import Ollama


Load the Vector Store

In [2]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

C:\Users\sachd\AppData\Local\Temp\ipykernel_30700\2191836911.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")


In [3]:
vectorstore = FAISS.load_local(
    "faiss_e5_index1",
    embedding,
    allow_dangerous_deserialization=True,
    index_name="index"  
)

In [4]:
llm = Ollama(model="llama3.2")

C:\Users\sachd\AppData\Local\Temp\ipykernel_30700\1555928118.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


Loading the Retriever

In [6]:
retriever = FAISS.load_local(
    "faiss_e5_index1",
    embedding,
    allow_dangerous_deserialization=True
).as_retriever(search_kwargs={"k": 10})

To improve the answer quality we will use re-ranking to get better results

In [7]:
co = cohere.ClientV2(os.environ.get("COHERE_API_KEY"))

In [8]:
def cohere_rerank(query, docs, top_n=5):
    doc_texts = [doc.page_content for doc in docs]
    response = co.rerank(query=query, documents=doc_texts, top_n=top_n, model="rerank-v3.5")
    reranked = [docs[r.index] for r in response.results]
    return reranked

In [9]:
def answer_with_rerank(query):
    retrieved_docs = retriever.get_relevant_documents(query)
    reranked_docs = cohere_rerank(query, retrieved_docs, top_n=5)
    context = "\n\n".join([doc.page_content for doc in reranked_docs])
    prompt = f"Answer the following legal question in detail using the context below:\n\nQuestion: {query}\n\nContext:\n{context}\n\nAnswer:"
    response = llm.invoke(prompt)
    return response

In [11]:
def answer_question(query,history):
    response = answer_with_rerank(query)
    return response

In [13]:

legal=gr.ChatInterface(
    fn=answer_question,
    title="Legal Question Answering",
    description="Ask a legal question and get an answer based on the provided context.",
    examples=[
        ["Can an employer change terms without notifying employees?"],
        ["What rights do tenants have if their landlord doesn’t make repairs?"],
        ["Can I challenge this arbitration clause in court?"]
    ]
)
legal.launch(share=True)

c:\Users\sachd\anaconda3\envs\llms\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://d3a4979c9ecf330f22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
